In [ ]:
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torch.optim import Adam
import numpy as np
import gymnasium as gym
from gymnasium.spaces import Discrete, Box

In [32]:
def mlp(sizes, activation=nn.Tanh, output_activation=nn.Identity):
    # Build a feedforward neural network.
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)

def train(env_name='CartPole-v0', hidden_sizes=[32], lr=1e-2, 
          epochs=50, batch_size=5000, render=False):

    # make environment, check spaces, get obs / act dims
    env = gym.make(env_name)
    assert isinstance(env.observation_space, Box), \
        "This example only works for envs with continuous state spaces."
    assert isinstance(env.action_space, Discrete), \
        "This example only works for envs with discrete action spaces."

    obs_dim = env.observation_space.shape[0]
    n_acts = env.action_space.n

    # make core of policy network
    logits_net = mlp(sizes=[obs_dim]+hidden_sizes+[n_acts])

    # make function to compute action distribution
    def get_policy(obs):
        logits = logits_net(obs)
        return Categorical(logits=logits)

    # make action selection function (outputs int actions, sampled from policy)
    def get_action(obs):
        return get_policy(obs).sample().item()

    # make loss function whose gradient, for the right data, is policy gradient
    def compute_loss(obs, act, weights):
        logp = get_policy(obs).log_prob(act)
        return -(logp * weights).mean()

    # make optimizer
    optimizer = Adam(logits_net.parameters(), lr=lr)

    # for training policy
    def train_one_epoch():
        # make some empty lists for logging.
        batch_obs = []          # for observations
        batch_acts = []         # for actions
        batch_weights = []      # for R(tau) weighting in policy gradient
        batch_rets = []         # for measuring episode returns
        batch_lens = []         # for measuring episode lengths

        # reset episode-specific variables
        obs,_ = env.reset()       # first obs comes from starting distribution
        done = False            # signal from environment that episode is over
        ep_rews = []            # list for rewards accrued throughout ep

        # render first episode of each epoch
        finished_rendering_this_epoch = False

        # collect experience by acting in the environment with current policy
        while True:

            # rendering
            if (not finished_rendering_this_epoch) and render:
                env.render()
                
            

            # save obs
            batch_obs.append(obs.copy())

            # act in the environment
            act = get_action(torch.as_tensor(obs, dtype=torch.float32))
            obs, rew, done, _,_ = env.step(act)

            # save action, reward
            batch_acts.append(act)
            ep_rews.append(rew)

            if done:
                # if episode is over, record info about episode
                ep_ret, ep_len = sum(ep_rews), len(ep_rews)
                batch_rets.append(ep_ret)
                batch_lens.append(ep_len)

                # the weight for each logprob(a|s) is R(tau)
                batch_weights += [ep_ret] * ep_len

                # reset episode-specific variables
                obs, done, ep_rews = env.reset(), False, []
                obs = obs[0]

                # won't render again this epoch
                finished_rendering_this_epoch = True

                # end experience loop if we have enough of it
                if len(batch_obs) > batch_size:
                    break

                    
        batch_obs = np.stack(batch_obs)       
        batch_acts = np.stack(batch_acts) 
        batch_weights = np.stack(batch_weights)
        
        # take a single policy gradient update step
        optimizer.zero_grad()
        batch_loss = compute_loss(obs=torch.as_tensor(batch_obs, dtype=torch.float32),
                                  act=torch.as_tensor(batch_acts, dtype=torch.int32),
                                  weights=torch.as_tensor(batch_weights, dtype=torch.float32)
                                  )
        batch_loss.backward()
        optimizer.step()
        return batch_loss, batch_rets, batch_lens

    # training loop
    for i in range(epochs):
        batch_loss, batch_rets, batch_lens = train_one_epoch()
        print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
                (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))



In [33]:
env_name = 'CartPole-v0'
#env = gym.make(env_name)
#obs,_ = env.reset() 
lr = 1e-2
render = 'store_false'

train(env_name=env_name, lr=lr)

epoch:   0 	 loss: 14.815 	 return: 17.203 	 ep_len: 17.203
epoch:   1 	 loss: 15.408 	 return: 18.621 	 ep_len: 18.621
epoch:   2 	 loss: 16.983 	 return: 19.748 	 ep_len: 19.748
epoch:   3 	 loss: 19.302 	 return: 21.805 	 ep_len: 21.805
epoch:   4 	 loss: 22.080 	 return: 25.125 	 ep_len: 25.125
epoch:   5 	 loss: 26.859 	 return: 28.874 	 ep_len: 28.874
epoch:   6 	 loss: 29.198 	 return: 32.128 	 ep_len: 32.128
epoch:   7 	 loss: 31.649 	 return: 34.696 	 ep_len: 34.696
epoch:   8 	 loss: 34.008 	 return: 39.754 	 ep_len: 39.754
epoch:   9 	 loss: 35.231 	 return: 44.439 	 ep_len: 44.439
epoch:  10 	 loss: 37.209 	 return: 48.295 	 ep_len: 48.295
epoch:  11 	 loss: 43.700 	 return: 52.884 	 ep_len: 52.884
epoch:  12 	 loss: 42.408 	 return: 53.511 	 ep_len: 53.511
epoch:  13 	 loss: 45.868 	 return: 61.378 	 ep_len: 61.378
epoch:  14 	 loss: 43.312 	 return: 57.273 	 ep_len: 57.273
epoch:  15 	 loss: 50.367 	 return: 65.455 	 ep_len: 65.455
epoch:  16 	 loss: 52.999 	 return: 69.2